# IR and raman

- IR-active modes are characterized by changes in the dipole moment of the molecule during vibration. To calculate IR intensities, you can use the following formula for each mode:


$
I_{IR} \propto \sum_{i} \left| \sum_{\alpha} \mathbf{e}_{i,\alpha} \cdot \mathbf{d}_{\alpha} \right|^2 \cdot \delta(\omega - \omega_i)
$



where $\mathbf{e}_{i,\alpha}$   is the eigenvector of mode$i$ along direction $\alpha$ ; $\mathbf{d}_{\alpha}$  is the unit vector for the 
$e_{i, α-th}$ Cartesian direction, a d $\delta(\omega-\omega_i)$ is a Dirac delta function centered at the frequency $\omega_i$ of the mode $i$


The resulting values represent the intensities of IR-active modes in the calculated IR spectrum.
- On the other side, to estimate  Raman Intensities
Raman-active modes are characterized by changes in polarizability during vibration. The Raman intensity can be calculated using the following formula for each mode:


$I_{\text{Raman}} \propto \sum_{i} \left| \sum_{\alpha, \beta} \mathbf{e}_{i,\alpha} \cdot \mathbf{e}_{i,\beta} \cdot \alpha_{\alpha, \beta} \right|^2 \cdot \delta(\omega - \omega_i$


where$ \alpha_{\alpha, \beta}$   is the polarizability tensor element between directio $ $
α an$d$ β.

The resulting values represent the intensities of Raman-active modes in the calculated Raman spectrum.

In [1]:
!pip install mp-api
!pip install mpcontribs-client
!pip install crystal-toolkit --upgrade
!pip install dash


  Using cached crystal_toolkit-2023.11.3-py3-none-any.whl (15.9 MB)


  Using cached crystaltoolkit-extension-0.6.0.tar.gz (2.9 MB)


  Installing build dependencies ... -

 \

 |

...

^C
 canceled
ERROR: Operation cancelled by user


  Using cached dash-2.15.0-py3-none-any.whl (10.2 MB)


  Using cached dash_html_components-2.0.0-py3-none-any.whl (4.1 kB)


  Using cached dash_core_components-2.0.0-py3-none-any.whl (3.8 kB)


  Using cached dash_table-5.0.0-py3-none-any.whl (3.9 kB)


In [1]:
from ipywidgets import interact, Dropdown
import numpy as np
from pymatgen.io.cif import CifWriter
import nglview as nv
import pandas as pd
from mp_api.client import MPRester
from pylab import *
import pandas as pd
from emmet.core.summary import HasProps
from nglview import show_structure_file
from pymatgen.io.cif import CifWriter

In [2]:
from mp_api.client import MPRester

with MPRester("A55hSVu7NuwvlISilTvVtw37IB5ES26w") as mpr:
    ph_bs = mpr.get_phonon_bandstructure_by_material_id("mp-149")

The server does not support the request made to https://api.materialsproject.org/materials/phonon/mp-149/?_limit=1&_fields=ph_bs. This may be due to an outdated mp-api package, or a problem with the query.


Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Document primary key has changed from mp-149 to mp-149, returning data for mp-149 in materials/phonon route.    


---------------------------------------------------------------------------
MPRestError                               Traceback (most recent call last)
File c:\Users\roste\anaconda3\lib\site-packages\mp_api\client\core\client.py:1001, in BaseRester.get_data_by_id(self, document_id, fields)
   1000 try:
-> 1001     results = self._query_resource_data(criteria=criteria, fields=fields, suburl=document_id)  # type: ignore
   1002 except MPRestError:

File c:\Users\roste\anaconda3\lib\site-packages\mp_api\client\core\client.py:958, in BaseRester._query_resource_data(self, criteria, fields, suburl, use_document_model, timeout)
    945 """Query the endpoint for a list of documents without associated meta information. Only
    946 returns a single page of results.
    947 
   (...)
    956     A list of documents
    957 """
--> 958 return self._query_resource(  # type: ignore
    959     criteria=criteria,
    960     fields=fields,
    961     suburl=suburl,
    962     use_document_model=us

# Example for $TiO_2 - rutile$

In the folowing example we took the resulting frequencies from the phonon dispersion at the so called $\Gamma$ point, which is the the symmetric point found at $\hat{R}= 0 \hat{h} + 0 \hat{l} + 0 \hat{k}$

In [3]:

# Here you can insert the materials project ID
material_id = "mp-2657"

# This API key can be requested in the website: https://next-gen.materialsproject.org/api
API_key     = "A55hSVu7NuwvlISilTvVtw37IB5ES26w"


def plot_phonon_and_structure(material_id):
    with MPRester(API_key) as mpr:
        ph_bs = mpr.get_phonon_bandstructure_by_material_id(material_id)
        ph_dos = mpr.get_phonon_dos_by_material_id(material_id)
        structure = mpr.get_structure_by_material_id(material_id)

    position = []
    label = []
    lines = []

    for i in ph_bs.as_phononwebsite()["line_breaks"]:
        lines.append(i[0])

    for i in ph_bs.as_phononwebsite()["highsym_qpts"]:
        position.append(i[0])
        label.append("$" + i[1] + "$")

    print(structure)
    cif_filename = f"material_structure_{material_id}.cif"  # Name of the output CIF file

    cif_writer = CifWriter(structure)
    cif_writer.write_file(cif_filename)
    struct_file = nv.FileStructure(cif_filename)
    
    view = nv.show_file(struct_file)
    view._remote_call("setSize", args=["", "300px"])
    view.camera = "orthographic"
    view.add_unitcell()
    view
    fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(15, 7), gridspec_kw={'wspace': 0.2, 'hspace': 0, 'width_ratios': [2, 1]})
    ax[0].plot(ph_bs.as_phononwebsite()["eigenvalues"])
    density_points = len(ph_dos.as_dict()["densities"])
    ax[0].set_ylim(0, (density_points))
    ax[0].set_xlim(0, max(position))
    # print("here : ", position, label)
    ax[0].set_xticks(ticks= position, labels=label)
    ax[0].set_ylabel(r"Frequency [$cm^{-1}$]")
    ax[0].set_xlabel(r"K  [$\frac{2\pi}{a}$]")
    ax[1].set_xlabel(r"DOS")
    for i in lines:
        ax[0].axvline(i - 1, c="black")

    ax[1].plot(ph_dos.as_dict()["densities"], np.arange(density_points))
    ax[1].set_ylim(0, (density_points))
    ax[1].set_xlim(0)
    return view
plot_phonon_and_structure(material_id)
    

The server does not support the request made to https://api.materialsproject.org/materials/phonon/mp-2657/?_limit=1&_fields=ph_bs. This may be due to an outdated mp-api package, or a problem with the query.


Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Document primary key has changed from mp-2657 to mp-2657, returning data for mp-2657 in materials/phonon route.    


---------------------------------------------------------------------------
MPRestError                               Traceback (most recent call last)
File c:\Users\roste\anaconda3\lib\site-packages\mp_api\client\core\client.py:1001, in BaseRester.get_data_by_id(self, document_id, fields)
   1000 try:
-> 1001     results = self._query_resource_data(criteria=criteria, fields=fields, suburl=document_id)  # type: ignore
   1002 except MPRestError:

File c:\Users\roste\anaconda3\lib\site-packages\mp_api\client\core\client.py:958, in BaseRester._query_resource_data(self, criteria, fields, suburl, use_document_model, timeout)
    945 """Query the endpoint for a list of documents without associated meta information. Only
    946 returns a single page of results.
    947 
   (...)
    956     A list of documents
    957 """
--> 958 return self._query_resource(  # type: ignore
    959     criteria=criteria,
    960     fields=fields,
    961     suburl=suburl,
    962     use_document_model=us

In [ ]:
import numpy as np

bands, qpoints, basis, vector = shape(ph_bs.eigendisplacements)
# Sample eigenvectors and eigenvalues (replace with your actual data)

eigenvectors = ph_bs.eigendisplacements
eigenvalues  = ph_bs.bands
qpoints_v = ph_bs.as_dict()["qpoints"]



# Constants (you might need to adjust these)
proportionality_IR = 1.0
proportionality_Raman = 1.0

# Calculate IR intensities
IR_intensities = np.zeros((bands, qpoints))
for i in range(bands):
    for q in range(qpoints):
        for basis_dir in range(basis):
            IR_intensities[i, q] += np.abs(np.dot(
                eigenvectors[i, q, basis_dir],
                qpoints_v[q])  # Replace with actual Cartesian direction vector
            ) ** 2

# Calculate Raman intensities
Raman_intensities = np.zeros((bands, qpoints))
for i in range(bands):
    # for q in range(qpoints):
    for basis_dir_alpha in range(basis):
        for basis_dir_beta in range(basis):
            Raman_intensities[i, 0] += np.abs(np.dot(
                        eigenvectors[i, 0, basis_dir_alpha],
                        eigenvectors[i, 0, basis_dir_beta]
                    )) ** 2 * proportionality_Raman

# Normalize intensities
IR_intensities *= proportionality_IR
Raman_intensities *= proportionality_Raman

# Now you can plot the IR and Raman spectra using your preferred plotting library
# (e.g., Matplotlib)
# Plot Raman Spectrum
plt.figure(figsize=(6, 6))
frequencies=eigenvalues
IR_total= list(zeros(len(frequencies[0])))
Freq_total= list(zeros(len(frequencies[0])))
for i in range(bands):
    plt.scatter(frequencies[i]*33.356, Raman_intensities[i], label=f'Mode {i+1}')
    IR_total+=list(Raman_intensities[i])
    Freq_total+=list(frequencies[i]*33.356)



Freq_0 = list(arange(0,2000,0.1))
IR_0   = list(zeros(len(Freq_0)))

Freq_total_s = Freq_0 + Freq_total
IR_total_s   = IR_0 + IR_total
all= []
for i in range(len(IR_total_s)):
    all.append(tuple((Freq_total_s[i],IR_total_s[i])))

all.sort()
x= []
y= []
for i in all:
    x.append(round(i[0],1))
    y.append(i[1])
    
xlabel(r"$Frequency [cm^{-1}]$")
ylabel(r"Intensity")

In [ ]:
frequencies = x 
intensities = y 


# Crear un diccionario para realizar un seguimiento de las sumas de intensidades por frecuencia
frequency_intensity_dict = {}


for frequency, intensity in zip(frequencies, intensities):

    if frequency in frequency_intensity_dict:
        frequency_intensity_dict[frequency] += intensity
    else:
       
        frequency_intensity_dict[frequency] = intensity

result_frequencies = list(frequency_intensity_dict.keys())
result_intensities = list(frequency_intensity_dict.values())

plot(result_frequencies,result_intensities)
xlabel(r"$Frequency [cm^{-1}]$")
ylabel(r"Intensity")
xlim(0,900)


In [ ]:
from scipy.signal import argrelextrema
def g(wavenumb_sweep, intensity_max, wavenumber_max, σ):
    G = intensity_max / (σ *sqrt(2 * pi)) * exp(-(wavenumb_sweep-wavenumber_max)**2 / (2*σ**2))
    new_y=array(G)  
    return new_y
result_frequencies = list(frequency_intensity_dict.keys())
result_intensities = list(frequency_intensity_dict.values())      

wavenumb   = result_frequencies
max_int    = max(result_intensities)

norm_int = []
for i in result_intensities:
    norm_int.append(i/max_int)

### Gausian function to broaden peaks
pos_max    = argrelextrema(array(norm_int), np.greater)
x          = array(wavenumb)
all_curve = 0
σ = 20
for i in pos_max[0]:
    all_curve += g(x, norm_int[i], wavenumb[i],σ)      
broad_int = all_curve   

### Normalization

max_y = max(broad_int)
int_norm = []

for i in broad_int:
    int_norm.append((i/max_y))
plot(wavenumb,int_norm)

plot(result_frequencies,result_intensities)
xlabel(r"$Frequency [cm^{-1}]$")
ylabel(r"Intensity")
xlim(0,900)


## Comparison of experimental data and simulated Raman

In [ ]:
from txt_raman_reader import RRUFF_text

In [ ]:
df=RRUFF_text("rutile_exp_dep.txt")._read_file()
plot(df["wavenumbers"], df["intensities_normalized"])
plot(wavenumb,int_norm)

xlabel(r"$Frequency [cm^{-1}]$")
ylabel(r"Intensity")
xlim(0,900)


# How Raman and IR are simulated in reality? 

In simulations of Raman and IR spectra within the framework of density functional theory (DFT), the calculation of Raman and IR intensities involves the polarizability tensor and Born charge derivatives:

**Polarizability Tensor:**

The polarizability tensor is a key concept in the calculation of Raman and IR intensities. It characterizes how the electron cloud of a molecule or crystal responds to an external electric field. The polarizability tensor is usually represented as a 3x3 matrix in three-dimensional space. Each element of this matrix represents how the electron cloud responds to an electric field in a specific direction.

In the context of Raman and IR spectroscopy:

- IR Spectroscopy: In IR spectroscopy, the polarizability tensor is used to calculate the intensity of IR-active modes. When atoms in a material vibrate, they induce changes in the electron density, leading to changes in the dipole moment of the system. The polarizability tensor relates the atomic displacements to these changes in dipole moment, allowing you to determine which vibrational modes will be IR-active.

- Raman Spectroscopy: In Raman spectroscopy, the polarizability tensor is used to calculate the intensity of Raman-active modes. Here, the change in the polarizability of the system due to atomic vibrations is what matters. The polarizability tensor relates the atomic displacements to changes in the polarizability, and this information helps determine which vibrational modes will be Raman-active.

**Born Charge Derivatives:**

The Born charge derivatives are another critical component in these simulations, especially when dealing with crystalline materials. The Born charge, denoted as *Z*, represents the effective charge of an atom in a crystal due to the interaction with the surrounding lattice. The Born charge derivatives refer to how this effective charge changes as the atomic positions vary.

In the context of Raman and IR simulations:

IR Spectroscopy: To calculate IR intensities in crystalline materials, you often need the derivatives of the Born charge with respect to atomic displacements. These derivatives, known as the "Born effective charges" help account for the response of atoms within the crystal to the vibrational modes. They are essential for determining the IR activity of vibrational modes in crystals.

Raman Spectroscopy: Born charge derivatives are also important for Raman spectroscopy, particularly in the context of lattice dynamics. These derivatives describe how the polarizability changes with atomic displacements and are used to calculate Raman intensities, especially in crystalline materials.

In summary, the polarizability tensor and Born charge derivatives play crucial roles in simulating Raman and IR spectra within the DFT framework, allowing you to predict which vibrational modes will be active and the intensities of these modes in the corresponding spectra. Understanding these quantities and their relationships to the atomic and electronic properties of the material is essential for accurate spectroscopic predictions.

In [ ]:


freq_dft = [1.129780207231549269e+02 , 1.129780207231552680e+02 ,1.749879864729470853e+02 ,1.749879864729474264e+02 ,3.611532269867757350e+02 ,4.793102600546089320e+02 ,4.957302896450344178e+02 ,6.001592985713667758e+02 ,6.001592985713667758e+02] 
int_dft = [1.000000000000000000e+00,9.999999999999938938e-01,2.939221091892205324e-03,2.939221091892158921e-03,1.494269659327405464e-01,1.890540997545240437e-02,1.019312927046264333e-01,1.445200115409593922e-01,1.445200115409594477e-01]

def Gaus_norm( list_freq,list_int, sigma):
    ### Gausian function to broaden peaks
    int_dft_m      = list_int + list(zeros(len(wavenumb)))
    x_dft          = list_freq + wavenumb
    
    all= []
    for i in range(len(int_dft_m )):
        all.append(tuple((x_dft[i],int_dft_m[i])))
    
    all.sort()
    x= []
    y= []
    for i in all:
        x.append(round(i[0],1))
        y.append(i[1])
    
    frequencies = x
    intensities = y
    
    # Crear un diccionario para realizar un seguimiento de las sumas de intensidades por frecuencia
    frequency_intensity_dict = {}
    
    for frequency, intensity in zip(frequencies, intensities):
    
        if frequency in frequency_intensity_dict:
            frequency_intensity_dict[frequency] += intensity
        else:
           
            frequency_intensity_dict[frequency] = intensity
    
    result_frequencies_dft = list(frequency_intensity_dict.keys())
    result_intensities_dft = list(frequency_intensity_dict.values())    
    
    
    wavenumb_dft   = result_frequencies_dft
    max_int    = max(result_intensities_dft)
    
    norm_int = []
    for i in result_intensities_dft:
        norm_int.append(i/max_int)
    
    ### Gausian function to broaden peaks
    pos_max    = argrelextrema(array(norm_int), np.greater)
    
    x          = array(wavenumb)
    
    all_curve = 0
    σ = sigma
    for i in pos_max[0]:
        all_curve += g(x, norm_int[i], wavenumb[i],σ)      
    broad_int = all_curve   
    
    ### Normalization
    
    max_y = max(broad_int)
    int_norm_dft = []
    
    for i in broad_int:
        int_norm_dft.append((i/max_y))
    return wavenumb, int_norm_dft, result_intensities_dft
import json
 
# Opening JSON file
f = open('crd.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)


In [ ]:

def intensity_raman(raman_tensor):
    """ Average a Raman-activity tensor to obtain a scalar
    intensity. """

    # This formula came from D. Porezag and M. R. Pederson, Phys. Rev.
    # B: Condens. Matter Mater. Phys., 1996, 54, 7830.
    
        
    if raman_tensor==None or raman_tensor==0:
        return 0
    else: 

        alpha = (
            (raman_tensor[0][0] + raman_tensor[1][1] + raman_tensor[2][2])
            / 3.0)
    
        beta_squared = 0.5 * (
            (raman_tensor[0][0] - raman_tensor[1][1]) ** 2
            + (raman_tensor[0][0] - raman_tensor[2][2]) ** 2
            + (raman_tensor[1][1] - raman_tensor[2][2]) ** 2
            + 6.0 * (raman_tensor[0][1] ** 2 + raman_tensor[0][2] ** 2 +
                raman_tensor[1][2] ** 2)
            )
    
        return (45.0 * alpha ** 2 + 7.0 * beta_squared)
freq = data["4146"]["key_value_pairs"]["frequencies_cm"]
freq= freq.replace("[", " ")
freq=freq.replace("]", " ")
freq=freq.replace("\n", " ")
freq=freq.split()
freqs_db = [float(i) for i  in freq ] 
ints_ram_t = []
for i in data["4146"]["data"]["raman_tensors"]:
    ints_ram_t.append(intensity_raman(i))
    
int_db = data["4146"]["data"]["Ramanactive"]["__ndarray__"][2]

# freq_gn, int_gn, int_n = Gaus_norm(freqs_db,int_db,10)
# plot(freq_gn, int_n, label= "DFT-DB-1")

freq_gn, int_gn, int_n = Gaus_norm(freqs_db,ints_ram_t,10)
plot(freq_gn, int_gn, label= "DFT-DB")
plot(df["wavenumbers"], df["intensities_normalized"])
xlabel(r"$Frequency [cm^{-1}]$")
ylabel(r"Intensity")
xlim(0,900)


# wavenumb, int_norm_dft, result_intensities_dft = Gaus_norm( freq_dft, int_dft, 10)

# Research at EPFL 

In [ ]:
df_a=RRUFF_text("anastase_532_exp.txt")._read_file()

freq_dft = [1.129780207231549269e+02 , 1.129780207231552680e+02 ,1.749879864729470853e+02 ,1.749879864729474264e+02 ,3.611532269867757350e+02 ,4.793102600546089320e+02 ,4.957302896450344178e+02 ,6.001592985713667758e+02 ,6.001592985713667758e+02] 
int_dft = [1.000000000000000000e+00,9.999999999999938938e-01,2.939221091892205324e-03,2.939221091892158921e-03,1.494269659327405464e-01,1.890540997545240437e-02,1.019312927046264333e-01,1.445200115409593922e-01,1.445200115409594477e-01]
freq_dft, int_dft, a = Gaus_norm(freq_dft,int_dft,20)
plot(freq_dft,int_dft)
plot(df_a["wavenumbers"],df_a["intensities_normalized"])

xlabel(r"$Frequency [cm^{-1}]$")
ylabel(r"Intensity")
xlim(0,900)
